In [146]:
import onnxruntime
import numpy as np
import torch
from torch.autograd import Variable
from onnxruntime.datasets import get_example
from PIL import Image
import dataset
import utils

In [147]:
example_model = get_example("/home/kobeyu/workspace/tmp/crnn/crnn.pytorch/crnn.onnx")
sess = onnxruntime.InferenceSession(example_model)

In [148]:
img_path = './data/demo.png'
ori_image = Image.open(img_path).convert('L')
print(type(ori_image))
transformer = dataset.resizeNormalize((100, 32))
image = transformer(ori_image)
print(type(image))
image.shape
image = image.view(1, *image.size()) 
print(type(image))
print(image.shape)
image = Variable(image)
print(type(image))
# image

<class 'PIL.Image.Image'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([1, 1, 32, 100])
<class 'torch.Tensor'>


In [149]:
input_name = sess.get_inputs()[0].name

print("Input name  :", input_name)
input_shape = sess.get_inputs()[0].shape
print("Input shape :", input_shape)
input_type = sess.get_inputs()[0].type
print("Input type  :", input_type)

Input name  : input.1
Input shape : [1, 1, 32, 100]
Input type  : tensor(float)


In [150]:
print(len(sess.get_outputs()))
output_name = sess.get_outputs()[0].name
print("Output name  :", output_name)  
output_shape = sess.get_outputs()[0].shape
print("Output shape :", output_shape)
output_type = sess.get_outputs()[0].type
print("Output type  :", output_type)


1
Output name  : 234
Output shape : [26, 1, 37]
Output type  : tensor(float)


In [151]:
%%time
result = sess.run([output_name], {input_name: image.numpy()})

CPU times: user 156 ms, sys: 4 ms, total: 160 ms
Wall time: 64.2 ms


In [152]:
result

[array([[[  4.77999449e-04,   3.87084763e-03,   4.78849523e-02,
           -2.79840007e-02,  -2.06379313e-02,  -5.11336289e-02,
           -3.80588323e-03,   3.84004340e-02,  -1.88542400e-02,
            5.27602397e-02,  -8.13531596e-03,  -4.09255251e-02,
            3.01768817e-03,   4.18673381e-02,  -2.28903443e-02,
            3.95095274e-02,  -2.74776146e-02,   4.28215228e-03,
           -3.55584882e-02,  -5.58080226e-02,  -7.84104504e-03,
           -4.19188151e-03,  -4.26546410e-02,   1.92571636e-02,
            6.16419353e-02,  -1.73552800e-03,  -9.61937383e-03,
            3.11060697e-02,   6.68683127e-02,   1.02468347e-03,
           -1.03103183e-02,   5.03733233e-02,  -4.19191867e-02,
           -4.57381755e-02,   5.24965208e-03,  -1.94062125e-02,
            1.18236952e-02]],
 
        [[  2.84698792e-04,   1.52158970e-03,   4.79634106e-02,
           -3.23849432e-02,  -1.74910724e-02,  -4.78081331e-02,
           -2.47740326e-03,   4.18015905e-02,  -2.10050866e-02,
        

In [153]:

result = np.reshape(result, (26,1,37))
# print(np.array(result).shape)
t_result = torch.tensor(result).float()
print(t_result.shape)
_, preds = t_result.max(2)
print(preds)
preds = preds.transpose(1, 0).contiguous().view(-1)
preds_size = Variable(torch.IntTensor([preds.size(0)]))

torch.Size([26, 1, 37])
tensor([[28],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24],
        [24]])


In [154]:
alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
converter = utils.strLabelConverter(alphabet)
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))                      

rnnnnnnnnnnnnnnnnnnnnnnnnn => rn                  
